In [1]:
#taken from here: https://github.com/lcbueno/NLP---Summary-with-Luhn-s-Algorithm/blob/main/nlp-summary-with-luhn-s-algorithm.ipynb
# https://github.com/pemagrg1/text_summarization/blob/master/for_python3/main.py

In [2]:
# import re  
import nltk  
# import string
# import heapq
!pip install sumy
from sumy.nlp.stemmers import Stemmer
import pandas
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

   
from sumy.utils import get_stop_words
from sumy.nlp.stemmers import Stemmer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer as sumytoken

from sumy.summarizers.luhn import LuhnSummarizer


def luhn_summarizer(text, stemmer, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, sumytoken(LANGUAGE))
    summarizer_luhn = LuhnSummarizer(stemmer)
    stop_words = get_stop_words(LANGUAGE)
    complete_stop_words = set(["Assessment", "Response", "Plan", "Action"])
    for i in stop_words:
      complete_stop_words.add(i)
    summarizer_luhn.stop_words = complete_stop_words
    sentences = []
    for sentence in summarizer_luhn(parser.document, SENTENCES_COUNT):
        a = sentence
        sentences.append(str(a))
    return " ".join(sentences)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
root_dir = "/content/gdrive/My Drive/masters_thesis/"
data_file = root_dir + "hadm_to_discharge.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
df = pandas.read_csv(data_file)
df.head()

,hadm_id,discharge_summary
0,174680.0,Admission Date: [**2823-9-29**] ...
1,117806.0,Admission Date: [**2830-3-12**] ...
2,184534.0,Admission Date: [**2714-3-12**] ...
3,195146.0,Admission Date: [**2678-10-4**] ...
4,154895.0,Admission Date: [**2936-5-6**] Discharge ...


In [6]:
LANGUAGE = "english"
SENTENCES_COUNT = 3
stemmer = Stemmer(LANGUAGE)

import csv

fieldnames = ["hadm_id", "source", "target", "discharge_summary"]
file_to_write = root_dir + "setup2_luhn_discharge.csv"
with open(file_to_write, 'w') as csvfile:
  csvwriter = csv.DictWriter(csvfile, delimiter=',', fieldnames=fieldnames)
  csvwriter.writerow(dict((fn,fn) for fn in fieldnames))
  test_array = []
  for index, row in df.iterrows():
    if (index == 9999):
      break
    source = row["discharge_summary"]
    hadm_id = row["hadm_id"]
    luhn_summary = luhn_summarizer(source, stemmer, LANGUAGE, SENTENCES_COUNT)
    test_array.append({"source": source, "target": luhn_summary, "hadm_id": hadm_id, "discharge_summary": source})
  for row in test_array:
    csvwriter.writerow(row)